In [ ]:
# Magic functions -- Run Once
%load_ext autoreload
%autoreload 2
%matplotlib notebook

# Move up one folder to reach the repo root
%cd ..

from utils.notebook.generic import full_width_notebook

full_width_notebook()

In [ ]:
# Paths, Imports & Configs
import matplotlib.pyplot as plt

from utils.file import read_json

from utils.generic import get_answer_to_family_map, chain_load_experiment_stats, separate_stats_by_set
#from utils.generic import chain_load_batch_metrics
from utils.notebook.result_analysis import load_experiment_predictions, sort_correct_incorrect_predictions
from utils.notebook.result_analysis import plot_confusion_matrix, plot_predictions_distribution_per_question_family
from utils.notebook.result_analysis import plot_predictions_confidence, plot_acc_loss_by_epoch, plot_predictions_confidence_gap
from utils.notebook.generic import separate_preds_ground_truth

root_data_path = "data"
root_output_path = "output_synced/training"

# No coordconv
folder_path = "CLEAR_50k_4_inst_audio_win_512_hop_2048_keep_256_norm_zero_one_norm_clear_stats_table_7_CoordConv_Extr_None_Stem_None_Resblock_None_Classifier_None_40_epoch_189369_table_7_table_7_final/2020-12-07_06h02"

# With only time coordconv
folder_path = "CLEAR_50k_4_inst_audio_win_512_hop_2048_keep_256_norm_zero_one_norm_clear_stats_table_7_CoordConv_Extr_None_Stem_None_Resblock_Time_Classifier_None_40_epoch_876944_table_7_table_7_final/2020-12-01_14h29"

# Resnet
folder_path = "CLEAR_50k_4_inst_audio_win_512_hop_2048_keep_256_RGB_norm_zero_one_norm_clear_stats_resnet_table_1_baseline_40_epoch_427438_table_1_table_1_final_final/2020-12-06_15h43"

# Test
folder_path = 'CLEAR_50k_4_inst_audio_win_512_hop_2048_keep_256_norm_zero_one_norm_clear_stats_extractor_slim_parallel_3_block_64_proj_spatial_time_resblock_classifier_40_epoch_876944_1d_spatial_test/2020-11-07_23h58'

# NAAQA
folder_path = 'CLEAR_50k_4_inst_audio_win_512_hop_2048_keep_256_norm_zero_one_norm_clear_stats_table_2_parallel_extractor_40_epoch_876944_table_2_table_2_final_final/2020-11-29_21h51'

folder_path = 'CLEAR_50k_4_inst_audio_win_512_hop_2048_mels_128_norm_zero_one_norm_clear_stats_table_7_bigger_interleaved_CoordConv_Extr_None_Stem_None_Resblock_Time_Classifier_None_40_epoch_876944_review_coordinate_maps_mel_part_1/2021-10-28_02h17'

experiment_output_path = 'output/inference/CLEAR_cogent_50k_4_inst_audio_test_cogent/2021-11-04_20h20'

experiment_output_path = 'output/inference/CLEAR_cogent_50k_4_inst_audio_test_cogent/2021-11-04_20h55'

#experiment_output_path = f"{root_output_path}/{folder_path}"
epoch_id = "best"

data_name = "CLEAR_cogent_50k_4_inst_audio"
data_path = f"{root_data_path}/{data_name}"


In [ ]:
all_filters = set()
for q in questions:
    if sum([1 for p in q['program'] if p['type'] == "relate"]) > 0:
        continue
        
    test1 = [n['type'] for n in q['program'] if 'filter' in n['type']]
    test2 = [n['type'] for n in q['program'] if 'filter' in n['type'] and len(n['value_inputs']) > 0]
    
    if len(test1) != len(test2):
        print(set(test1) - set(test2))
        
    filters = {n['type'] for n in q['program'] if 'filter' in n['type']}
    all_filters.update(filters)
    
all_filters

In [ ]:
{q['template_index'].split('-')[0] for q in questions if ' or ' in q['question']}

In [ ]:
[q for q in [qo for qo in with_or if qo['program'][-1]['type'] == 'or']]

In [ ]:
bool_q = [questions[p['question_id']] for p in predictions if p['ground_truth_answer_family'] == 'boolean']
{q['program'][-1]['type'] for q in bool_q}

In [ ]:
{q['question'] for q in questions if 'count' in q['template_index'] and q['program'][-1]['type'] == 'count_different_instrument'}

In [ ]:
experiment_output_path

In [ ]:
sum([1 for p in predictions if p['correct']])/len(predictions)

In [ ]:
answer_to_family_map
questions[0]
#get_answer_to_family_map(f'{data_path}/attributes.json', to_lowercase=True)

In [ ]:
# Load data from files
answer_to_family_map = get_answer_to_family_map(f'{data_path}/attributes.json', to_lowercase=True, reduced_text=True)
answer_families = list(set(answer_to_family_map.values()))

predictions = load_experiment_predictions(experiment_output_path, epoch_id, set_type='test', reduced_text=True)
questions = read_json(f"{data_path}/questions/CLEAR_test_questions.json")['questions']

for pred in predictions:
    question = questions[pred['question_id']]
    
    pred['relate_count'] = sum([1 for p in question['program'] if p['type'] == "relate"])
    pred['position_filter'] = len([1 for p in question['program'] if "filter_position" in p['type']]) > 0
    pred['has_relate'] = pred['relate_count'] > 0
    
import pandas as pd

predictions_df = pd.DataFrame(predictions, columns=['question_id', 'scene_id', 'scene_length', 'correct', 'prediction', 'ground_truth', 'prediction_answer_family', 'ground_truth_answer_family', 'confidence', 'relate_count', 'has_relate', 'position_filter'])
grouped = predictions_df.groupby(['ground_truth_answer_family','correct','relate_count', 'position_filter'], as_index=False)

In [ ]:
# NOTE : THIS IS DEFINITELY NOT THE GOOD WAY OF DOING THIS... JUST NEED SOME QUICK DATA

families = {k[0] for k in grouped.groups.keys()}

grouped_count = grouped.count()

agg = []
for family in families:
    global_correct_count = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == True)]['prediction'].sum()
    global_incorrect_count = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == False)]['prediction'].sum()
    
    correct_count_no_relate = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == True) & (grouped_count['relate_count'] == 0)]['prediction'].sum()
    incorrect_count_no_relate = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == False) & (grouped_count['relate_count'] == 0)]['prediction'].sum()
    
    correct_count_one_relate = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == True) & (grouped_count['relate_count'] == 1)]['prediction'].sum()
    incorrect_count_one_relate = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == False) & (grouped_count['relate_count'] == 1)]['prediction'].sum()
    
    correct_count_two_relate = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == True) & (grouped_count['relate_count'] == 2)]['prediction'].sum()
    incorrect_count_two_relate = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == False) & (grouped_count['relate_count'] == 2)]['prediction'].sum()
        
    if correct_count_two_relate + incorrect_count_two_relate > 0:
        acc_two_relate = correct_count_two_relate / (correct_count_two_relate + incorrect_count_two_relate)
    else:
        acc_two_relate = pd.NA
        
    correct_with_filter_position = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == True) & (grouped_count['position_filter'] == True)]['prediction'].sum()
    incorrect_with_filter_position = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == False) & (grouped_count['position_filter'] == True)]['prediction'].sum()
    
    correct_no_filter_position = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == True) & (grouped_count['position_filter'] == False)]['prediction'].sum()
    incorrect_no_filter_position = grouped_count[(grouped_count['ground_truth_answer_family'] == family) & (grouped_count['correct'] == False) & (grouped_count['position_filter'] == False)]['prediction'].sum()
    
    agg.append({
        'family' : family,
        'acc_global': global_correct_count / (global_correct_count + global_incorrect_count),
        'acc_with_filter_position': correct_with_filter_position / (correct_with_filter_position + incorrect_with_filter_position),
        'acc_no_filter_position': correct_no_filter_position / (correct_no_filter_position + incorrect_no_filter_position),
        'acc_no_relate': correct_count_no_relate / (correct_count_no_relate + incorrect_count_no_relate),
        'acc_one_relate': correct_count_one_relate / (correct_count_one_relate + incorrect_count_one_relate),
        'acc_two_relate': acc_two_relate,
        
    })
    
df = pd.DataFrame(agg).sort_values('family', ascending=False)

from utils.notebook.pandas import color_by_multi_attribute

colored_df = color_by_multi_attribute(df, main_attribute="family", 
                                 attributes=['acc_global', 'acc_with_filter_position', 'acc_no_filter_position', 'acc_no_relate', 'acc_one_relate', 'acc_two_relate'], 
                                 cmaps=['Blues','ocean_r','ocean_r','ocean_r','ocean_r','ocean_r','ocean_r'])

colored_df

In [ ]:
cols = {'acc_no_relate': 'No relation', 'acc_one_relate': '1 relation', 'acc_two_relate': '2 relation'}
df[list(cols.keys())].mean(axis=0).rename(cols).plot.bar(rot=0)

In [ ]:
# No coordinate maps -- Parallel feature extractor
colored_df

In [ ]:
# Time coordinate maps in resblocks only -- Parallel feature extractor
colored_df

In [ ]:
# Coordinate maps everywhere (Both) -- Resnet feature extractor
colored_df